In [1]:
%%capture

import warnings

warnings.filterwarnings("ignore")
import _ct_district_grain_data_prep as _ct_district_data_prep
import branca
import branca.colormap as cm
import _report_utils
import calitp_data_analysis.magics
import geopandas as gpd
import pandas as pd
from great_tables import GT
from IPython.display import HTML, Image, Markdown, display, display_html
from slugify import slugify
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS
from shared_utils import webmap_utils

In [2]:
import google.auth
credentials, project = google.auth.default()
import gcsfs

fs = gcsfs.GCSFileSystem()

In [3]:
# Comment out and leave this cell right below pandas
# district = "06 - Fresno / Bakersfield"
# district = "09 - Bishop"
district = "07 - Los Angeles / Ventura"

In [4]:
# %%capture_parameters
# district

In [5]:
#DISTRICT_DIGEST_URL = (
#    "https://gtfs-digest--cal-itp-data-analyses.netlify.app/district_"
#  f"{slugify(district_str)}"
# )

In [6]:
# district_str = str(district)

In [7]:
# Extract district from district string when doing an sjoin
# between CT districts & routes
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [8]:
# Files for webmaps
boundary_file = f"district_{district_int}_boundary"
transit_routes_file = f"district_{district_int}_transit_routes"
shn_file = f"district_{district_int}_shn"
transit_shn_file = f"district_{district_int}_transit_routes_shn"

In [9]:
# Load Datasets
operator_df = _ct_district_data_prep.data_wrangling_operator_profile(district)
operator_list = list(operator_df.analysis_name.unique())

In [10]:
operator_gdf = _ct_district_data_prep.data_wrangling_operator_map(
    list(operator_df.analysis_name.unique())
)

In [11]:
ct_district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [13]:
transit_route_shs_gdf, transit_route_shs_table = (
    _ct_district_data_prep.final_transit_route_shs_outputs(15, district_int)
)

In [14]:
district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [15]:
gtfs_table_df = _ct_district_data_prep.create_gtfs_stats(operator_df)

In [16]:
shn_gdf = _ct_district_data_prep.load_buffered_shn_map(district_int)

# District {district}

These are district summaries for [GTFS Digest](https://gtfs-digest--cal-itp-data-analyses.netlify.app/). 

In [17]:
#Individual transit operators have their pages at: **[{DISTRICT_DIGEST_URL}]#({DISTRICT_DIGEST_URL})**

In [18]:
district_summary = _report_utils.district_stats(operator_df, "caltrans_district")

In [19]:
summary_table1 = (
    GT(
        district_summary.drop(columns=["arrivals_per_stop", "trips_per_operator"]).pipe(
            _report_utils.transpose_summary_stats, district_col="caltrans_district"
        )
    )
    .fmt_number(columns="value", sep_mark = ",")
    .cols_label(index="")
    .tab_header(title=f"District {district} GTFS summary stats")
)

summary_table2 = (
    GT(
        district_summary[
            ["caltrans_district", "arrivals_per_stop", "trips_per_operator"]
        ].pipe(_report_utils.transpose_summary_stats, district_col="caltrans_district")
    )
    .fmt_number("value", decimals=1, sep_mark = ",")
    .cols_label(index="")
)

In [20]:
display(summary_table1)
display(summary_table2)

GT(_tbl_data=         index      Value
0  # Operators         52
1     # routes        805
2      # trips     32,062
3      # stops     30,322
4   # arrivals  1,476,197, _body=<great_tables._gt_data.Body object at 0x78f26866bf10>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='Value', type=<ColInfoTypeEnum.default: 1>, column_label='Value', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x78f26866b610>, _spanners=Spanners([]), _heading=Heading(title='District 07 - Los Angeles / Ventura GTFS summary stats', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x78f227f91150>, _formats=[<great_tables._gt_data.FormatInfo object at 0x78f227f90850>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, category='heading', type='px', value='125%'), heading_title_font_weight=OptionsInfo(scss=True, category='heading', type='value', value='initial'), headin

,Value
Arrivals per Stop,48.68
Trips per Operator,616.58


## Routes within the District

In [21]:
color_map = cm.linear.Spectral_11.scale()

In [22]:
color_map = branca.colormap.LinearColormap(
    colors=color_map.colors[7:], vmin=0, vmax=operator_gdf.Number.max()
)

In [23]:
district_map = webmap_utils.set_state_export(
    district_gdf,
    subfolder = "caltrans_district_digest/",
    filename=boundary_file,
    map_title="District Map",
    overwrite = True
)

In [24]:
transit_routes = webmap_utils.set_state_export(
    operator_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_routes_file,
    map_title="Transit Routes",
    cmap=color_map,
    color_col="Number",
    existing_state=district_map,
    overwrite = True
    
)

In [25]:
webmap_utils.render_spa_link(transit_routes["spa_link"], text="Open Routes for all Operators Map")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJEaXN0cmljdCBNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2NhbHRyYW5zX2Rpc3RyaWN0X2RpZ2VzdC9kaXN0cmljdF83X2JvdW5kYXJ5Lmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19LCB7Im5hbWUiOiAiVHJhbnNpdCBSb3V0ZXMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2NhbHRyYW5zX2Rpc3RyaWN0X2RpZ2VzdC9kaXN0cmljdF83X3RyYW5zaXRfcm91dGVzLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzQuMDgwNzI0NzQyNTg0MTI0LCAtMTE3Ljc1MjAzMzQ1ODEyOTM5XSwgInpvb20iOiAxM30=" target="_blank">Open Open Routes for all Operators Map in New Tab</a>

In [26]:
webmap_utils.display_spa_map(transit_routes["spa_link"])

## Transit Routes on the State Highway Network
**Only transit routes that have 15% or more if its length on one or more State Highway Network routes are included**

In [27]:
color_map2 = cm.linear.RdYlBu_11.scale()
# adjust the number of steps as needed

In [28]:
color_map2 = branca.colormap.LinearColormap(
    colors=color_map2.colors[7:], vmin=0, vmax=100
)

In [29]:
shn_map = webmap_utils.set_state_export(
    shn_gdf,
    subfolder = "caltrans_district_digest/",
    filename=shn_file,
    map_title="State Highway Network Map",
    map_type='state_highway_network',
    overwrite = True
)

In [30]:
# transit_route_shs_gdf.drop(columns = ["geometry"]).sample()

In [31]:
transit_shn_map = webmap_utils.set_state_export(
    transit_route_shs_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_shn_file,
    map_title="Transit Routes on the State Highway Network",
    cmap=color_map2,
    color_col="Percentage of Transit Route on SHN Across All Districts",
    existing_state=shn_map,
    legend_url="https://storage.googleapis.com/calitp-map-tiles/transit_route_pct.svg",
    overwrite = True
    
)

In [32]:
webmap_utils.render_spa_link(transit_shn_map["spa_link"], text="Open Routes on State Highway System Map")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTdGF0ZSBIaWdod2F5IE5ldHdvcmsgTWFwIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy9jYWx0cmFuc19kaXN0cmljdF9kaWdlc3QvZGlzdHJpY3RfN19zaG4uZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fSwgInR5cGUiOiAic3RhdGVfaGlnaHdheV9uZXR3b3JrIn0sIHsibmFtZSI6ICJUcmFuc2l0IFJvdXRlcyBvbiB0aGUgU3RhdGUgSGlnaHdheSBOZXR3b3JrIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy9jYWx0cmFuc19kaXN0cmljdF9kaWdlc3QvZGlzdHJpY3RfN190cmFuc2l0X3JvdXRlc19zaG4uZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNC4yMjg5OTg0NzE0NzkzOCwgLTExOC4zODY1NTQ3MzMwMDg3N10sICJ6b29tIjogMTMsICJsZWdlbmRfdXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3RyYW5zaXRfcm91dGVfcGN0LnN2ZyJ9" target="_blank">Open Open Routes on State Highway System Map in New Tab</a>

In [33]:
webmap_utils.display_spa_map(transit_shn_map["spa_link"])

In [34]:
GT(
    transit_route_shs_table.sort_values(
        by=[
            "Analysis Name",
            "Percentage of Transit Route on SHN Across All Districts",
        ],
        ascending=[True, False],
    )
)

Analysis Name,Route,State Highway Network Routes in District 7,Percentage of Transit Route on SHN Across All Districts
Antelope Valley Transit Authority,Route 790,"138, 14",90.6
Antelope Valley Transit Authority,Route 785,"134, 2, 110, 118, 405, 5, 210, 101, 14, 170",82.7
Antelope Valley Transit Authority,Route 787,"118, 405, 5, 210, 101, 14",63.4
Antelope Valley Transit Authority,Route 786,"118, 405, 5, 210, 101, 14, 138",60.0
Antelope Valley Transit Authority,Route 52,138,26.8
Antelope Valley Transit Authority,Route 2,"138, 14",23.3
Antelope Valley Transit Authority,Route 1,"138, 14",21.2
City of Calabasas,Line 5,101,34.1
City of Commerce,600 Transit Center-Citadel Outlets-Dowtown L.A.,"60, 110, 5, 101, 10, 710",63.4
City of Gardena,2 Line 2,"213, 405, 105, 91, 1",25.5


## GTFS Stats by Operator

In [35]:
string_cols = gtfs_table_df.select_dtypes(include="object").columns.tolist()

In [36]:
gtfs_table = (
    GT(gtfs_table_df.sort_values("# Trips", ascending=False))
    .fmt_integer(
        columns=[
            c
            for c in gtfs_table_df.columns
            if c not in ["Operator Service Miles", "Avg Arrivals per Stop"]
            and (c not in string_cols)
        ]
    )
    .fmt_number(columns=["Operator Service Miles", "Avg Arrivals per Stop"], decimals=1)
    .data_color(
        columns=["# Trips", "Avg Arrivals per Stop"],
        palette=["#e0ecf6", "#376fa9"],
        na_color="lightgray",
    )
    .tab_header(
        title=f"District {district}",
        subtitle="Daily GTFS schedule statistics by operator",
    )
    .cols_align(
        columns=[
            c
            for c in gtfs_table_df.columns
            if c not in ["Organization", "Transit Operator"]
        ],
        align="center",
    )
)

In [37]:
gtfs_table = _report_utils.great_table_formatting(gtfs_table)
gtfs_table

GT(_tbl_data=                                        Analysis Name  # Routes  # Trips  \
46  Los Angeles County Metropolitan Transportation...       114    14513   
25                                City of Los Angeles        66     3128   
43                                   Foothill Transit        35     2101   
44                                 Long Beach Transit        36     1891   
36                               City of Santa Monica        17     1538   
51  Ventura County (VCTC, Gold Coast, Cities of Ca...        57     1142   
42                              FlixBus and Greyhound       213      801   
14                                City of Culver City        11      666   
0                   Antelope Valley Transit Authority        20      554   
45                                 Los Angeles County        18      495   
39                                   City of Torrance        12      443   
35                              City of Santa Clarita        38      426   
47                         Los Angeles World Airports         8      425   
31                                   City of Pasadena        11      417   
28                                 City of Montebello         8      394   
19                                   City of Glendale        11      367   
30                                    City of Norwalk         6      315   
18                                    City of Gardena         6      287   
49        Southern California Regional Rail Authority         7      210   
26                                    City of Lynwood         8      143   
16                                   City of El Monte         7      127   
48           Palos Verdes Peninsula Transit Authority        10      122   
12                                   City of Commerce         7      120   
50              University of California, Los Angeles         4      113   
15                                     City of Downey        12      104   
1                                    City of Alhambra         2      101   
8                                     City of Burbank         2       99   
32                              City of Redondo Beach         2       91   
21                            City of Huntington Park         2       90   
20                                   City of Glendora         5       89   
2                                     City of Arcadia         3       89   
41                             City of West Hollywood         3       65   
40                                City of West Covina         3       64   
38                                 City of South Gate         2       64   
29                              City of Monterey Park         4       62   
24                                   City of Lawndale         2       60   
4                                City of Baldwin Park         5       49   
11                                   City of Cerritos         6       42   
7                                  City of Bellflower         2       40   
6                                City of Bell Gardens         1       30   
33                                   City of Rosemead         2       28   
23                                  City of La Puente         2       26   
9                                   City of Calabasas         5       19   
34                               City of San Fernando         1       18   
5                                        City of Bell         1       18   
10                                     City of Carson         2       16   
27                                    City of Maywood         1       16   
13                                     City of Cudahy         1       12   
3                                     City of Artesia         1       11   
37                               City of Sierra Madre         1        8   
17                                 City of El Segundo         1        7   
22                                 